# Electric Vehicles in Washington State


## Introduction

The purpose of this notebook is to work with statistics on electrical vehicles (EVs) published in Washington State, using official records from the Department of Licensing of that state. The level of adoption of these vehicles is shown by county, as a proportion of EVs by 1,000 inhabitants, and that information is related to air quality measurements. This set of visualizations can help to:

(a) Observed differences among counties related to the adoption of EVs (b) Elaborate questions on why those differences exist among counties (c) Promote public discussion about how incentives related to the transition to EVs should be address in Washington State

Vehicles with internal combustion engines are among the main causes of greenhouse gas (GHG) emissions. EVs are seen as a promising alternative to reduce those emissions due to they don't produce CO2. Moreover, energy used to charge EVs can come from renewable sources, that can expand benefits for the environment.

Governments and the private sector are promoting the adoption of EVs to accelerate a transition towards a more sustainable transportation system. Public policies adopted include purchase financial incentives, charging infrastructure investment, low or zero emission vehicle mandates, and research and development. Knowledge about how adoption of EVs is happening can contribute to address more effectively those policies and to promote more incentives.

## Situation

The State of Washington Department of Licensing keep records of EVs and publishes a full dataset about that. That office has available to the public three visualizations:

* [Electric vehicle population by postal code](https://data.wa.gov/Demographics/Electric-Vehicle-Population-Map-by-Postal-Code/bhmw-igtj): It is a map of the United States that show markers over postal codes with the number of electric vehicles registered there. Most vehicles reported by this map are in Washington State, because records correspond to that state. This visualization includes controls to filter the data by county, city, maker, model, electric range, CAFV elegibility, type of vehicle, legislative district, postal code, and state.

* [Electric vehicles by county](https://data.wa.gov/Demographics/Electric-Vehicles-By-County/smxa-ttv3): It is a bar chart that represents the number of electric vehicles register for every county in a selected state (the default option is Washington State). The visualization includes controls to filter the data by county, vehicle type, maker, model, and state.

* [Top 10 electric vehicle models](https://data.wa.gov/Demographics/Top-10-Electric-Vehicle-Models/cki8-rxms): It is a pie chart that represents the number of vehicles for the 10 top categories. Numbers are expresed in absolute terms and models are differentiated by color. The visualization has controls to filter the data by type of vehicle, county, city, state, maker, legislative district, and model year.

These visualizations are useful to have a first impression about the electric vehicles sample included in this dataset. However, they present limitations as:

* While data by county and ZIP code helps to quantify EVs adoption in absolute terms, such condition is subject to the amount of population that every county has. Therefore, in addition to the absolute number of vehicles, it is useful to have a relationship between this data and the number of inhabitants.

* On the other hand, since one of the main contributions expected from the adoption of EVs is the reduction of pollution and the emission of greenhouse gases, it is important to also have a notion about the air quality in each county. It would be expected that in those areas with poorer levels in terms of air quality, the promotion for the adoption of EVs would be more sustained.

## Goal

The goal of this project is:

> To visualize differences among counties in Washington State related to the adoption of electrical vehicles (EVs) in relationship with the number of inhabitantes and the levels of air quality.

## Tasks

Tasks implemented in this project were:

1. Exploratory data analysis: the dataset is loaded and its main characteristics are identified
2. Data transformations: data is aggregated by county and merged with population and air quality statistics
3. Visualizations: Preliminary visualizations and the main one are prepared, with a summary of key elements used in the design
4. Discussion: an evaluation with a panel of users and results
5. Findings: A synthesis of the findings of this project

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import altair as alt
import vega_datasets

## Dataset

### Description

The dataset used in this project is about Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs). Records refer to vehicles registered by the State of Washington Department of Licensing. Data was downloaded from [Data.gov](https://catalog.data.gov/dataset/electric-vehicle-population-data) on Apr. 29, 2023, and it is used under the terms of the [Open Data Commons Open Database License (OBbL) v1.0](https://opendatacommons.org/licenses/odbl/1-0/).

### Attributes

* VIN: Unique vehicle identification code
* Country: County where the vehicle is registered
* City: City where the vehicle is registered
* State: State where the vehicle is registered
* Postal Code: Postal code of the zone where the vehicle is registered
* Model Year: Year of the vehicle's model
* Make: Name of the maker
* Model: Model name
* Electric Vehicle Type: type of vehicle (BEV or PHEV)
* Clean Alternative Fuel Vehicle (CAFV) Elegibility: Indication if the vehicle meets criteria to be elegible as Clean Alternative Fuel Vehicle
* Electric Range: Voltage of operation
* Base MSRP: Suggested price
* Legislative District: District where the vehicle is registered
* DOL Vehicle ID: Identification assigned by the Department of Licensing
* Vehicle Location: geographical point where the vehicle is registered
* Electric utility: Utility company the serves the area where the vehicle is registered
* 2020 Census Tract: Census tract where the vehicle is registered

### Loading the Dataset

In this section the dataset is loaded into memory. Its main characteristics are explored.

In [2]:
data = pd.read_csv("Electric_Vehicle_Population_Data.csv")
data.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJ3E1EB4L,Yakima,Yakima,WA,98908.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,322,0,14.0,127175366,POINT (-120.56916 46.58514),PACIFICORP,5.307700e+10
1,5YJ3E1EA7K,San Diego,San Diego,CA,92101.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220,0,NaN,266614659,POINT (-117.16171 32.71568),NaN,6.073005e+09
2,7JRBR0FL9M,Lane,Eugene,OR,97404.0,2021,VOLVO,S60,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,22,0,NaN,144502018,POINT (-123.12802 44.09573),NaN,4.103900e+10
3,5YJXCBE21K,Yakima,Yakima,WA,98908.0,2019,TESLA,MODEL X,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,289,0,14.0,477039944,POINT (-120.56916 46.58514),PACIFICORP,5.307700e+10
4,5UXKT0C5XH,Snohomish,Bothell,WA,98021.0,2017,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,14,0,1.0,106314946,POINT (-122.18384 47.8031),PUGET SOUND ENERGY INC,5.306105e+10


In [3]:
# Counting the number of examples and attributes
examples, attributes = data.shape
print("Number of attributes: ", attributes)
print("Number of examples: ", examples)

Number of attributes:  17
Number of examples:  124716


In [4]:
# Showing the columns, data type, and number of data points
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124716 entries, 0 to 124715
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         124716 non-null  object 
 1   County                                             124714 non-null  object 
 2   City                                               124714 non-null  object 
 3   State                                              124716 non-null  object 
 4   Postal Code                                        124714 non-null  float64
 5   Model Year                                         124716 non-null  int64  
 6   Make                                               124716 non-null  object 
 7   Model                                              124535 non-null  object 
 8   Electric Vehicle Type                              124716 non-null  object

In [5]:
# Counting missed dat
data.isna().sum()

VIN (1-10)                                             0
County                                                 2
City                                                   2
State                                                  0
Postal Code                                            2
Model Year                                             0
Make                                                   0
Model                                                181
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         0
Base MSRP                                              0
Legislative District                                 297
DOL Vehicle ID                                         0
Vehicle Location                                      29
Electric Utility                                     473
2020 Census Tract                                      2
dtype: int64

In [6]:
# Looking for duplicated records
data.duplicated().sum()

0

### General Characteristics

* The dataset contains 124,716 examples with 17 attributes.
* Most attributes are categorical, except some like `Electric Range` and `Base MSRP`
* There are missed data for `Electric Utility`, `Legislative District`, `Model`, `Vehicle Location`, and `2020 Census Tract`, but that omited points are not significative due they are under 1%.
* There are not duplicated examples.

### Adjusment of Types

It was observed that for some attributes the data type is wrong. In this section the data type is adjusted in a way to represent in a better manner the data representation.

In [7]:
cols_to_fix_type = [
    'Postal Code',
    'Legislative District',
    'DOL Vehicle ID',
    '2020 Census Tract'
]
for column in cols_to_fix_type:
    data[column] = data[column].astype(str)

### Complementary Datasets

To do relationships between the number of electrical vehicles by county, two more datasets are used, one related to population and the other one about air quality.

* [Population estimates for Washington State](https://ofm.wa.gov/washington-data-research/population-demographics/population-estimates/historical-estimates-april-1-population-and-housing-state-counties-and-cities), provided by Washington State Office of Financial Management.
* [Air Quality Index](https://enviwa.ecology.wa.gov/Report/LatestAQI), obtained from Washington's Air Monitoring Program.

## Exploratory Data Analysis

The purpose of this section is to gain a deeper knowledge about the dataset, exploring the data represented by every attribute.

### Categorical Attributes

In [8]:
# Categorical attributes
attr_cat = [column for column in data.columns if data[column].dtype == 'O']
attr_cat

['VIN (1-10)',
 'County',
 'City',
 'State',
 'Postal Code',
 'Make',
 'Model',
 'Electric Vehicle Type',
 'Clean Alternative Fuel Vehicle (CAFV) Eligibility',
 'Legislative District',
 'DOL Vehicle ID',
 'Vehicle Location',
 'Electric Utility',
 '2020 Census Tract']

In [9]:
# The first ten categories are shown
for attr in attr_cat:
    print(f"##### {attr} (%) #####", '\n')
    counts = data[attr].value_counts()
    counts = counts / counts.sum() * 100
    n_items = len(counts)
    n_items = n_items if n_items <= 10 else 10
    print(counts[:n_items])
    print()

##### VIN (1-10) (%) ##### 

5YJYGDEE9M    0.371243
5YJYGDEE0M    0.368838
5YJYGDEE7M    0.363225
5YJYGDEE8M    0.352802
5YJYGDEE2M    0.345585
5YJYGDEEXM    0.340774
7SAYGDEEXN    0.339972
5YJYGDEE3M    0.336765
5YJYGDEE6M    0.335963
7SAYGDEE8N    0.334360
Name: VIN (1-10), dtype: float64

##### County (%) ##### 

King         52.334141
Snohomish    11.274596
Pierce        7.647898
Clark         5.936783
Thurston      3.646744
Kitsap        3.326010
Whatcom       2.486489
Spokane       2.469651
Benton        1.257277
Skagit        1.127379
Name: County, dtype: float64

##### City (%) ##### 

Seattle      17.647578
Bellevue      5.203105
Redmond       3.725324
Vancouver     3.579390
Kirkland      3.145597
Bothell       3.088667
Sammamish     2.929102
Renton        2.497715
Olympia       2.438379
Tacoma        2.128871
Name: City, dtype: float64

##### State (%) ##### 

WA    99.761859
CA     0.064146
VA     0.028866
MD     0.021649
TX     0.014433
CO     0.008018
NC     0.007216
AZ   

### Numerical Attributes

In [10]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Model Year,124716.0,2019.406339,2.976174,1997.0,2018.0,2020.0,2022.0,2023.0
Electric Range,124716.0,79.471936,100.331969,0.0,0.0,25.0,200.0,337.0
Base MSRP,124716.0,1556.068909,10053.289929,0.0,0.0,0.0,0.0,845000.0


### Detailed Characteristics

From the previous sections here is commented what are the most important charectistics observed when the dataset is explored by attribute.

* `VIN (1-10)` and `DOL Vehicle ID` are identification attributes, unique for every example. Therefore they are not relevant for an aggregated analysis.
* Most vehicles registered correspond to Washington State (99.76%). Vehicles from other states are under-represented. This makes senses due the office responsible to mantain the records is an statal branch.
* It has been confirmed that only BEVs and PHEVs vehicles are contained in the dataset; 77.2% are BEVs.
* Most vehicles are served by Puget Sound Energy and City of Seattle/Tacome utility companies (over 70%).
* Model years are from 1997 to 2023. At least 75% are from 2018.
* Although not missed values were reported for `Base MSRP`, it is noted that most values are zero. That can be an indicative that for most examples that value is not available.

### Adjusting the Dataset

From the previous observations, it was noticed that `VIN (1-10)` and `DOL Vehicle ID` attributes are not relevant for aggregated analysis. In consequence, they will be dropped from the dataset.

Besides that, given that information for other states is not representative, those examples will be removed. In that way, the dataset will only reflect examples of Washington State.

In [11]:
data.drop(columns=['VIN (1-10)', 'DOL Vehicle ID'], inplace=True)
data = data[data['State'] == 'WA']

## Data Transformations

In order have data available for the visualizations above indicated, it needed to load complementary datasets and perform aggregations by county.

### EVs and Population

In [12]:
# Loading population data

# Reading Excel file
population = pd.read_excel('popden_city.xlsx', sheet_name='City_Town', skiprows=5)
# Selecting relevant attributes
population = population[["City/Town Name", "County Name", "Population 2022"]]
# Renaming columns
population.rename(columns={
    "City/Town Name": "City",
    "County Name": "County",
}, inplace=True)
population.head()

,City,County,Population 2022
0,Aberdeen,Grays Harbor,17040
1,Airway Heights,Spokane,11040
2,Albion,Whitman,545
3,Algona,King,3300
4,Almira,Lincoln,320


In [13]:
# Grouping data by county and merging dataset

# Grouping data by county
by_county = data.groupby(["County"]).count()['State'].reset_index()
# Renaming columns
by_county.rename(columns={'State': 'Electrical Vehicles'}, inplace=True)
# Grouping population by county
pop = population.groupby('County').sum(numeric_only=True).reset_index()
# Merging with population data
ev_pop = pop.merge(by_county, on=["County"], how="left")
# Computing EV / population index
ev_pop["EV_Pop_1K"] = ev_pop["Electrical Vehicles"] / ev_pop["Population 2022"] * 1000
ev_pop = ev_pop.sort_values(by="EV_Pop_1K", ascending=False).dropna().round()
ev_pop.head()

,County,Population 2022,Electrical Vehicles,EV_Pop_1K
31,San Juan,2680,764.0,285.0
38,Wahkiakum,560,44.0,79.0
17,Jefferson,10290,763.0,74.0
26,Mason,10430,595.0,57.0
33,Skamania,2525,141.0,56.0


### EVs and Air Quality

In [14]:
# Loading air quality data

aqi = pd.read_excel("AQI Current Hour Summary.xlsx", sheet_name="Sheet1", skiprows=2)
aqi.head()

,County,Site,AQI Level,Pollutant,Occured,AQI
0,Adams,Ritzville-Alder St,Good,PM2.5,4/29/2023 05:00 PM,18
1,Asotin,Clarkston-13th St,Good,PM2.5,4/29/2023 05:00 PM,25
2,Benton,Kennewick-Metaline,Good,PM10,4/29/2023 05:00 PM,32
3,Benton,Kennewick-S Clodfelter Rd,Moderate,O3,4/29/2023 05:00 PM,71
4,Benton,Prosser-Highland Dr,Good,PM2.5,4/29/2023 05:00 PM,33


In [15]:
# Grouping air quality data by county
# Whether there more that one mearuments by county, the median is taken
aqi_by_county = aqi.groupby('County').median(numeric_only=True)['AQI']
aqi_by_county = pd.DataFrame(aqi_by_county).reset_index()
aqi_by_county.head()

,County,AQI
0,Adams,18.0
1,Asotin,25.0
2,Benton,33.0
3,Chelan,21.5
4,Clallam,28.0


In [16]:
# Merging data with air quality indices
ev_aqi = aqi_by_county.merge(by_county, on="County", how="left")
# Including qualitative descriptor
ev_aqi['Label'] = ev_aqi['AQI'].apply(lambda v: 
    'Good' if v <=50 else
    'Moderated' if v <= 100 else
    'Unhealty' if v <= 300 else
    'Hazardous')
ev_aqi.head()

,County,AQI,Electrical Vehicles,Label
0,Adams,18.0,35,Good
1,Asotin,25.0,52,Good
2,Benton,33.0,1567,Good
3,Chelan,21.5,732,Good
4,Clallam,28.0,779,Good


In [17]:
ev_aqi.Label.unique()

array(['Good'], dtype=object)

In all Washington state counties air quality is good, so the describite attribute will not be used.

## Visualizations

In this section, visualizations are presented. Preliminary displays are presented on the relationship between number of EVs and population by one side, and the other between number of EVs and air quality. After that, a unique visualization is integrated in line with the goal of this project.

### EV by 1,000 Inhabitants

In [24]:
pop_chart = (alt.Chart(ev_pop, title="Electrical Vehicles by 1,000 Inhabitants")
    .mark_bar(
        tooltip=True,
    )
    .encode(
        alt.X('County:N', sort="-y"),
        alt.Y('EV_Pop_1K:Q', title="EV by 1,000 inhabitants")
    ))

pop_chart

alt.Chart(...)

### EV vs Air Quality

In [22]:
chart = (alt.Chart(ev_aqi, title="EV vs air quality")
    .mark_point()
    .encode(
        alt.X("Electrical Vehicles:Q", title="Number of EVs", scale=alt.Scale(type="log")),
        alt.Y("AQI:Q", title="AQI (greater number is worst)"),
        alt.Tooltip("County:N"),
        alt.Color("County", scale=alt.Scale(scheme='spectral'), legend=None)
    )
    .interactive())
chart

alt.Chart(...)

This chart shows that there is a trend of major number of EV in places where the air quality is worst.

### Main Visualization

In [20]:
df = ev_pop.merge(ev_aqi.drop(columns="Electrical Vehicles"), on="County").sort_values('EV_Pop_1K', ascending=False)
df['size'] = df['EV_Pop_1K'] / 1000
df.head()

,County,Population 2022,Electrical Vehicles,EV_Pop_1K,AQI,Label,size
0,Jefferson,10290,763.0,74.0,23.0,Good,0.074
1,Mason,10430,595.0,57.0,18.0,Good,0.057
2,Island,27880,1402.0,50.0,10.0,Good,0.050
3,Kitsap,98860,4148.0,42.0,22.0,Good,0.042
4,King,1939870,65268.0,34.0,29.0,Good,0.034


In [21]:
selection = alt.selection(type='multi', fields=['County'])

pop_chart = (alt.Chart(df, title="EVs by 1,000 inhabitants")
    .properties(width=300, height=400)
    .mark_bar(
        tooltip=True,
    )
    .encode(
        alt.X('County:N', sort="-y"),
        alt.Y('EV_Pop_1K:Q', title="EV by 1,000 inhabitants"),
        color='County:N'
    )).add_selection(selection)

aqi_pop = (alt.Chart(df, title="EVs vs Air Quality")
    .properties(width=300, height=400)
    .mark_point(
        tooltip=True
    )
    .encode(
        alt.X("Electrical Vehicles:Q", title="Number of EVs", scale=alt.Scale(type="log")),
        alt.Y("AQI:Q", title="AQI (greater number is worst)"),
        alt.Tooltip("County:N"),
        size=alt.Size('size:Q', title="Proportion EV/1,000 inhab"),
        color=alt.Color('County:N', legend=None),
        fill=alt.Fill('County:N', legend=None)
    )).transform_filter(selection)

chart = pop_chart | aqi_pop

chart

alt.HConcatChart(...)

### Summary

Here, considerations related to the visualization design:
    
* The visualization has two sections. The first one is a bar chart (Chart I) where number of EVs by population are show by county in descendant order. The second one (Chart II) is a scatter plot that shows the relationship between the number of EVs and levels of air quality by county. The last chart also represents the proportion of EVs by population as the size of each point. In both plots, every county is shown in a different color.
* Chart I is a direct comparison by counties, to demonstrate in which counties the adoption of EVs is higher on one side, and on the other, in which counties that adoption is lower.
* Chart II includes multiple dimensions and requires a technical explanation about what AQI index represents and how air quality levels should be interpreted. A logarithmic scale is used to represent the number of EVs.
* As interactivity, a county can be selected in Chart I, filtering the result to Chart II. In that way, a more focused analysis can be realized by every county.

## Discussion

The main visualization was presented to three, neither domain experts nor visualization experts. They were asked about their interpretation of the main visualization. In summary, their responses were:

* Chart I is easy to interpret. In Jefferson County, there are more than 70 EVs by every 1,000 inhabitants. Colors help to differentiate every county, so it's easy to see in what counties there are a greater density of EVs related to population.
* Related to Chart 2, users ask how air quality is measure. There was some confusion because AQI scale is inverted, i.e. lower values mean a better air quality and higher values mean the worst air quality. After a short explanation about AQI indices, users don't found problem on interpreting the chart. It was noted that there is a trend in which counties with worst air quality have a greater density of EVs related to people living there.
* The functionality of choosing a county in Chart I and selecting that point in Chart II was positively valued in the sense it helps to characterize the situation for a selected county.

In general, users agree that this visualization is an improvement related to visualizations published by Washington State Department of Licensing, because they let make relationship between key variables related about the problem of GHG emissions and how the transition to EVs is occurring.

## Insight

This project shows how visualizations can help to represent a problematic aspect of reality and actions taken to improve the situation. In this case, the transition to EVs is an effort in which multiple stakeholders, like government bodies, private sector, and car owners, are taking part. Visualizations helps to identify patterns or trends, compare variables, and communicate insights. All this can contribute to identify intervention areas where policies and behaviors can be more effective. In this case, the visualization helps to compare the state of to-EVs transition among counties. It can be used to identify what actions are taking place in some counties that can help in others to accelerate the EVs transition. Overall, that can contribute to raise more public concern about GHG emissions and measure to reduce them.